In [1]:
import re
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dr_training_dataset = pd.read_csv('../Datasets/disaster_response_messages_training.csv', encoding = "ISO-8859-1", dtype={"split": "string", "message": "string", "original": "string", "genre": "string"})

In [3]:
general_tweet_dataset = pd.read_csv('../Datasets/general_tweet_dataset_reduced.csv',encoding = "ISO-8859-1", dtype={"split": "string", "message": "string", "original": "string", "genre": "string"})                                           

In [4]:
general_tweet_dataset.dtypes

ï»¿id                       int64
split                      string
message                    string
original                   string
genre                      string
related                   float64
PII                       float64
request                   float64
offer                     float64
aid_related               float64
medical_help              float64
medical_products          float64
search_and_rescue         float64
security                  float64
military                  float64
child_alone               float64
water                     float64
food                      float64
shelter                   float64
clothing                  float64
money                     float64
missing_people            float64
refugees                  float64
death                     float64
other_aid                 float64
infrastructure_related    float64
transport                 float64
buildings                 float64
electricity               float64
tools         

In [5]:
# checking that all records have a message value
print('Number of missing messages in message text:')
general_tweet_dataset['message'].isnull().sum()

Number of missing messages in message text:


0

In [6]:
# converting all float columns to int64
cols = general_tweet_dataset.columns[general_tweet_dataset.dtypes.eq('float64')]
general_tweet_dataset[cols] = general_tweet_dataset[cols].apply(np.int64, errors='coerce')

In [7]:
general_tweet_dataset.dtypes

ï»¿id                      int64
split                     string
message                   string
original                  string
genre                     string
related                    int64
PII                        int64
request                    int64
offer                      int64
aid_related                int64
medical_help               int64
medical_products           int64
search_and_rescue          int64
security                   int64
military                   int64
child_alone                int64
water                      int64
food                       int64
shelter                    int64
clothing                   int64
money                      int64
missing_people             int64
refugees                   int64
death                      int64
other_aid                  int64
infrastructure_related     int64
transport                  int64
buildings                  int64
electricity                int64
tools                      int64
hospitals 

In [8]:
# converting the int64 nulls to 2 (translates to no)
general_tweet_dataset = general_tweet_dataset.replace(-9223372036854775808,2)

In [9]:
general_tweet_dataset.sample(10)

,ï»¿id,split,message,original,genre,related,PII,request,offer,aid_related,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
10699,0,train,Studying for a final...too bad it's beautiful ...,Studying for a final...too bad it's beautiful ...,social,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
13458,0,train,ur beta off with her... im beta off without u....,ur beta off with her... im beta off without u....,social,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
3881,0,train,@bradiewebbstack aww poor bradie stuff those ...,@bradiewebbstack aww poor bradie stuff those ...,social,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
10207,0,train,no more easter eggs left,no more easter eggs left,social,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
6747,0,train,@clickokaynow No The kitties didn't make it out.,@clickokaynow No The kitties didn't make it out.,social,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
6861,0,train,Day two of training a tech who doesn't even kn...,Day two of training a tech who doesn't even kn...,social,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
5870,0,train,@Hamperlady Innocent no longer have my custom,@Hamperlady Innocent no longer have my custom,social,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
5072,0,train,aaa migraine poo k time to go to school i sle...,aaa migraine poo k time to go to school i sle...,social,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
10043,0,train,@grub00 Alas no: is a &quot;buy the MP3s&quot;...,@grub00 Alas no: is a &quot;buy the MP3s&quot;...,social,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
5286,0,train,why i can`t find dear friend i need her to tw...,why i can`t find dear friend i need her to tw...,social,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2


In [10]:
general_tweet_dataset.to_csv('../Datasets/⁩general_tweet_dataset_cleaned.csv', index=False) 

In [92]:
dr_training_dataset.sample(10)

,ï»¿id,split,message,original,genre,related,PII,request,offer,aid_related,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
7579,10590,train,"We're in rain season, people lives under shelt...",Nous sommes en pleines saisons pluievieuses. L...,direct,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
11961,17611,train,Preliminary report received from Tajikabad Red...,<NA>,news,1,0,0,0,0,...,0,1,1,1,0,0,0,0,0,0
18728,27006,train,"""Without proper sanitation, refugees already w...",<NA>,news,1,0,0,0,1,...,0,0,1,0,0,0,1,0,0,0
10144,14987,train,My tree about to fall da fukkk.. Sandy http://...,<NA>,social,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18727,27005,train,Mealy bug and mosaic viral disease in cassava ...,<NA>,news,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8993,12862,train,"I have winter clothing ( hats , scarves , glov...",<NA>,direct,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
14499,21137,train,Humanitarian sources told IRIN today that rail...,<NA>,news,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6675,9296,train,Is it possible for that the ONG in carfour(por...,Nap nande si li posib pou nan sitiyasyon sa a ...,direct,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8008,11294,train,RT Alyssa_Milano The Red Cross needs your help...,<NA>,social,1,0,1,0,1,...,0,0,1,0,0,0,1,0,0,0
14364,20946,train,"""Thanks to the incredibly generous contributio...",<NA>,news,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
dr_training_dataset.drop()